In [0]:
import os
import numpy as np
import matplotlib.pyplot as plt
import re
from google.colab import files

from keras.models import Sequential 
from keras.layers import Dense
from keras.layers import LSTM 
from keras.callbacks import LambdaCallback 
from keras import metrics

import os


In [0]:
# Load data into x and y for RNN, and give maps to and from chars
def parse_chars(text, seq_len, n_skip=1):
    # Convert text to list of limes
    lines = text.split('\n')
    
    # Define text_r as the reduced text variable, which is a single string containing all chars in order
    text_r = ""
    # For every line
    for line in lines:
        # If the length is greater than 1 (skipping the only new-line lines)
        if len(line) >= 1:
            # If there is no number in the line (skipping all the lines with the numbers)
            if not any(char.isdigit() for char in line):
                # Remove the double space at the beginning of certain lines
                if line[0] == ' ' and line[1] == ' ': 
                    line = line[2:]
                # Add the line to the string
                text_r = text_r + '\n' + line
    #text_r = text_r.lower()
    # For counting the number of unique characters
    n_chars = 0
    # For converting from characters to indices
    char_map = {}
    # For converting from indices 
    char_map_r = {}
    # For each character in our reduced text string
    for char in (text_r):
        # If the character is not already contained in the maps, add it
        if char not in char_map:
            char_map[char] = n_chars 
            char_map_r[n_chars] = char
            # Add one unique character
            n_chars += 1
        
    # Seqs takes every n_skip'th sequence to convert to x for training
    seqs = []
    # Takes the following character after seqs to convert to y for training
    seq_next = []
    
    # Create list of 40-character sequences, starting with every n_skip'th character.
    # Also store the following character
    for i in range(0, len(text_r) - seq_len, n_skip):
        seqs.append(text_r[i:i+seq_len])
        seq_next.append(text_r[i+seq_len])
        
    # The number of sequences
    N_seq = len(seqs)

    # Defining x and y arrays
    x = np.zeros((N_seq, seq_len, n_chars))
    y = np.zeros((N_seq, n_chars))
    
    for i, seq in enumerate(seqs):
        # Set the corresponding following character index to 1
        y[i, char_map[seq_next[i]]] = 1
        # Set each character in the sequence to 1 in its corresponding position (j)
        for j, char in enumerate(seq):
            x[i, j, char_map[char]] = 1
    
    return x, y, char_map, char_map_r

uploaded = files.upload()

In [0]:
text = open(os.path.join(os.getcwd(), 'shakespeare.txt')).read()
# Define the length of character sequence of interest
seq_len = 40
# Define the n^th character to skip
n_skip = 1
# Parse the characters into trainable data
x, y, char_map, char_map_r = parse_chars(text, seq_len, n_skip)
# Count the number of unique characters
n_chars = len(char_map)

print(char_map)

{'\n': 0, 'F': 1, 'r': 2, 'o': 3, 'm': 4, ' ': 5, 'f': 6, 'a': 7, 'i': 8, 'e': 9, 's': 10, 't': 11, 'c': 12, 'u': 13, 'w': 14, 'd': 15, 'n': 16, ',': 17, 'T': 18, 'h': 19, 'b': 20, 'y': 21, "'": 22, 'g': 23, 'v': 24, 'B': 25, 'p': 26, 'l': 27, 'H': 28, ':': 29, '-': 30, 'M': 31, 'k': 32, 'A': 33, 'W': 34, 'P': 35, '.': 36, 'z': 37, ';': 38, 'I': 39, 'S': 40, 'x': 41, 'L': 42, 'N': 43, 'D': 44, '?': 45, 'O': 46, 'C': 47, 'U': 48, 'q': 49, 'j': 50, 'R': 51, 'Y': 52, '(': 53, ')': 54, 'G': 55, 'V': 56, 'E': 57, 'K': 58, '!': 59, 'J': 60}


In [0]:
# Define the model as sequential
model = Sequential()
# Add an LSTM with 128 units, and input shape corresponding to the sequence length 
# and the number of unique characters
model.add(LSTM(192, input_shape=(seq_len, n_chars))) 
# Define dense layer output with softmax activation
model.add(Dense(n_chars, activation='softmax'))
# Define the model optimizer as RMSProp and the loss as categorical cross-entropy
model.compile(optimizer='rmsprop',metrics=['acc'],loss='categorical_crossentropy')

In [0]:
def sample(pred, temp=1.0):
    # Credit to fchollet script posted on PIazza
    # Take the predictions and convert them to an array of floats
    pred = np.asarray(pred).astype('float64')
    # Find the z_i values by taking the log, and divide by the temperatuer
    pred = np.log(pred) / temp
    # Take the exponent of the new probabilities, and divide them by the sum of probabilities
    pred = np.exp(pred) / np.sum(np.exp(pred))
    # Select the most-likely next character from this 
    return np.argmax(np.random.multinomial(1,pred,1))
                     
def on_epoch_end(epoch, _):
    if epoch % 10 == 0: 
      # Credit to fchollet script posted on piazza
      print()
      print('........ After Epoch %d ........' % epoch)
      # The length of the poem we wish to write following our first line
      poem_len = 600
      # For these 3 temperatuers
      for temp in [1.5, 0.75, 0.25]: 
        print('................................')
        print('........ Temperature %.1f ........' % temp)
        # The first line of interest
        sentence = "Shall I compare thee to a summer's day?\n"
        # Print the line, don't add another new line
        print(sentence,end='')
        # For the length of the poem 
        for i in range(poem_len):
            # Get the corresponding x-values for the preceding characters we wish to 
            x_pred = np.zeros((1,seq_len, n_chars))
            for j, char in enumerate(sentence):
                x_pred[0, j, char_map[char]] = 1
            
            # Predict the output probabilities
            pred = model.predict(x_pred, verbose=0)[0]
            # Use these probabilities with the given temperature to find the 
            # index of the next character
            next_idx = sample(pred,temp)
            # Retrieve the next character from this index
            next_char = char_map_r[next_idx]
            
            # Adjust the sentence to maintain its length, but encorporate the new character
            sentence = sentence[1:] + next_char
            # Print the new characters 1-by-1
            print(next_char,end='')
        print()
        print('................................')

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)


model.fit(x,y,batch_size=64,epochs=50,callbacks=[print_callback])

        

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/50





93634/93634 [==============================] - 131s 1ms/step - loss: 2.4257 - acc: 0.3188

........ After Epoch 0 ........
................................
........ Temperature 1.5 ........
Shall I compare thee to a summer's day?
Thyeiu'd I 'hery Acey uepath agkinatc noushdWalen.
du'sDy.
heoiclciIusouthiflid,tI thtu  gpspe:.
ond koyest or ard oucs bnd ben,vlendmocg diga:
Thy wocu the stfi' enkftmey.
,ha put le sle ewrena'aD.itulich fernotk foth kerss wefr oe yateO wevend rtkewesj
yeswbhindisimy's ane I da to I the groget,
Yit int-wrinite siyershogc nnlowrs nscigtind'ev bero
I, bithors chyvured, watd whrield to s welfselNt'stilli-couet
tiWhaswath eveonbro by sill ore cosi'se, geeisufegs aixly, neth,
Wenpeplrns myasns
' the,
 op dc vestls ficr t iwhadencbend lesee.
B iethinnit gre silk munake, chorts sogcly main,i, mo Al
................................
.......................

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


 of thee.
If thou bear fair feartake have and hapest those,
When all these paptarious past by mange,
And all my hage of self addanting heavterest know,
Though thou might I lest wrinkle know'st bake,
O would have entaIned then as I shane,
And sweaty adus trust report to die
ve say,
I doth times to willed and chance,
Ip 
................................
................................
........ Temperature 0.2 ........
Shall I compare thee to a summer's day?
The warrs it to the reast to the cance,
And they to summer's live the can all mand,
And for a swear her face is true doth lie,
That thou art fair from the proud of thee,
That thou bear failts with heaven the rist,
Then thought the praise them for my love seear,
For I am not ever my thee mind eyes,
That have by name the linured bainten shade,
And therefore mayst in their praise the sleare,
That heaven's are the world will beauty decay,
The cossess to be of state of the day,
And therefore to be ibsest that is in,
And love not seem the 

In [0]:
  sentence = "Shall I compare thee to a summer's day?\n"
  # Print the line, don't add another new line
  print(sentence,end='')
  poem_len = 800
  # For the length of the poem 
  for i in range(poem_len):
    # Get the corresponding x-values for the preceding characters we wish to 
    x_pred = np.zeros((1,seq_len, n_chars))
    for j, char in enumerate(sentence):
      x_pred[0, j, char_map[char]] = 1
            
    # Predict the output probabilities
    pred = model.predict(x_pred, verbose=0)[0]
    # Use these probabilities with the given temperature to find the 
    # index of the next character
    next_idx = sample(pred,0.25)
    # Retrieve the next character from this index
    next_char = char_map_r[next_idx]
            
    # Adjust the sentence to maintain its length, but encorporate the new character
    sentence = sentence[1:] + next_char
    # Print the new characters 1-by-1
    print(next_char,end='')


Shall I compare thee to a summer's day?
Thou art more blassed reaths of true mindsts,
And in their parts of make the sweetest be.
To thi

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


s that for my self, no bebled now,
And for their praise therefore to be belong,
Than when I partounes write of youns all,
What it me tongues a proudest of thy self,
So long as youth and thought that live and deed,
ot shand excupse that the will be wear,
That I in thee to live eye as eyes doth shand
To stands of pursused say I say not live,
And due to be, our changer beauty showe,
They love hore, not so greet thee first with despite,
The canker twaste, therefore to be belove.
Thou mary blass might I thow that befored,
And sweet some it but simpow restare some,
When you love's discimed and his scord of wronght,
And some theevil for a women's fast right,
And may by lip than cheek the lovely gran
Al

In [0]:
  sentence = "Shall I compare thee to a summer's day?\n"
  # Print the line, don't add another new line
  print(sentence,end='')
  poem_len = 800
  # For the length of the poem 
  for i in range(poem_len):
    # Get the corresponding x-values for the preceding characters we wish to 
    x_pred = np.zeros((1,seq_len, n_chars))
    for j, char in enumerate(sentence):
      x_pred[0, j, char_map[char]] = 1
            
    # Predict the output probabilities
    pred = model.predict(x_pred, verbose=0)[0]
    # Use these probabilities with the given temperature to find the 
    # index of the next character
    next_idx = sample(pred,0.75)
    # Retrieve the next character from this index
    next_char = char_map_r[next_idx]
            
    # Adjust the sentence to maintain its length, but encorporate the new character
    sentence = sentence[1:] + next_char
    # Print the new characters 1-by-1
    print(next_char,end='')


Shall I compare thee to a summer's day?
Thou art more from my farthess graces are spent.
Than they of such their propiling he reffalow.
To day that poor his image ade the past,
of true hath found of loves, you althering, and sunstagetty,
At still neve pay that in a thought to rehild.
This that repert welf welled with to be,
And that usI shor not better thee thingly night,
And ours in lover's shong a feed, of least,
of still every thal in his morrain age,
To will in thee this gall, I defear die.
If thou survive their spirit of their rehise,
Then I hase cridned of a worthings strange,
They with his bebuned how that sage of ill,
That in her pose pleased thought from thee,
Whi

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


le he isst praise him his istrace to main,
And that it worth even back then dightcy,
For shall now will my eye gowery yeth,
And by a perfect forgot unon aman.


In [0]:
  sentence = "Shall I compare thee to a summer's day?\n"
  # Print the line, don't add another new line
  print(sentence,end='')
  poem_len = 800
  # For the length of the poem 
  for i in range(poem_len):
    # Get the corresponding x-values for the preceding characters we wish to 
    x_pred = np.zeros((1,seq_len, n_chars))
    for j, char in enumerate(sentence):
      x_pred[0, j, char_map[char]] = 1
            
    # Predict the output probabilities
    pred = model.predict(x_pred, verbose=0)[0]
    # Use these probabilities with the given temperature to find the 
    # index of the next character
    next_idx = sample(pred,1.5)
    # Retrieve the next character from this index
    next_char = char_map_r[next_idx]
            
    # Adjust the sentence to maintain its length, but encorporate the new character
    sentence = sentence[1:] + next_char
    # Print the new characters 1-by-1
    print(next_char,end='')


Shall I compare thee to a summer's day?
Your inurestimy doth that I callue.
Then aumeh mistard hrave , might by theirsearnyvits,
Singl sumier watred with burbies ne due,
That in they , less have prearour in any'st,
't spendy numm aflove of thine thou carts make Will,
When thiugh ncees would goe, dver worsd aid,
lespetion be, a dignivest utol the sumnors,
Of this huicers and his scyined shows I live,
Appan I the the batincqued's dwell mera:
From shampered herp, tyonegh regimed best,
co, I'el fame eyes If thee, when groan seef:
O teast your wilt, nor for wo donsjuace,
 year feals primily sping,
When ip in hum in him behold a ftwersting,
lay spirit of eye's device more vainigued,
In prove hear that deas rigats than beyon,
Samethe hiseming summerts yours, worhings beartored,
It righing tre givet for this gentles fair,
tard torgmed a